In [1]:
import pandas as pd
import numpy as np
answers = pd.read_csv('./data/Answer.csv')
survey_2020 = pd.read_csv('./data/2020-survey.csv')

In [22]:
survey_2020 = survey_2020.columns[survey_2020.columns.str.startswith('q')]
survey_2020

AttributeError: 'Index' object has no attribute 'columns'

## Filter out unrelevant questions

In [2]:
answers = answers[answers.QuestionID < 13]

## Create survey DataFrame

We want a data frame where each row will represent one users answers to one survey. We create a empty DataFrame and fill it out with one row for each unique UserID found in the answers table.

In [3]:
surveys = pd.DataFrame(columns=['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'Year', 'UserID'])
user_ids = answers['UserID'].unique()
surveys['UserID'] = user_ids

In order to fill out the survey DataFrame we have to iterate through each row in the answers table and set the values in the corresponding row in surveys.

In [12]:
for answer in answers.iterrows():
    user_id = answer[1][2]
    question_id = 'q' + str(answer[1][3])
    answer_text = answer[1][0]
    year = answer[1][1]
    surveys.at[user_id - 1, question_id] = answer_text
    surveys.at[user_id - 1, 'Year'] = year
        
    

In [16]:
surveys.sample(20)

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,Year,UserID
2726,29,-1,Canada,-1,1,I don't know,1,-1,-1,-1,-1,No,2017,2727
1508,36,Male,United States of America,Arizona,0,No,0,More than 1000,0,No,I don't know,No,2016,1509
2923,25,Female,United States of America,Indiana,0,I don't know,1,26-100,1,I don't know,I don't know,No,2017,2924
730,37,Male,United Kingdom,-1,0,No,1,6-25,1,No,Don't know,Maybe,2014,731
1504,33,Male,Germany,-1,1,No,0,-1,-1,-1,-1,Maybe,2016,1505
3466,37,Male,United States of America,Florida,0,I don't know,1,More than 1000,1,Yes,I don't know,No,2018,3467
3253,27,Female,United States of America,Illinois,0,I don't know,0,100-500,1,Yes,I don't know,No,2017,3254
1233,23,Male,United Kingdom,-1,0,No,1,6-25,1,No,Don't know,No,2014,1234
1822,22,Male,United States of America,California,0,No,0,More than 1000,1,I don't know,I don't know,Maybe,2016,1823
2421,38,Male,United States of America,Pennsylvania,0,No,0,100-500,1,I don't know,I don't know,Yes,2016,2422


In [2]:
questions = pd.read_csv('./data/Question.csv')
questions.rename(columns={'questionid': 'QuestionID'}, inplace=True)
questions

,questiontext,QuestionID
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5
...,...,...
100,Do you think that team members/co-workers woul...,114
101,"If yes, what condition(s) have you been diagno...",115
102,"If maybe, what condition(s) do you believe you...",116
103,Which of the following best describes your wor...,117


In [3]:
surveys = pd.read_csv('./data/Survey.csv')
surveys

,SurveyID,Description
0,2014,mental health survey for 2014
1,2016,mental health survey for 2016
2,2017,mental health survey for 2017
3,2018,mental health survey for 2018
4,2019,mental health survey for 2019


## Negative age values
Some age values are negative, there fore those entries will be changed to positive.

In [10]:
answers.loc[answers['QuestionID'] == 1, 'AnswerText'] = answers.AnswerText.str.replace('-','')
sum([True for idx,row in answers.iterrows() if any(row.isnull())])

0

In [12]:
survey_answers = pd.merge(answers, questions, how="inner", on="QuestionID")
survey_answers = survey_answers.drop("QuestionID", 1)
has_duplicates = survey_answers['UserID'].duplicated().any()
print(has_duplicates)
survey_answers.head(200)

True


,AnswerText,SurveyID,UserID,questiontext
0,37,2014,1,What is your age?
1,44,2014,2,What is your age?
2,32,2014,3,What is your age?
3,31,2014,4,What is your age?
4,31,2014,5,What is your age?
...,...,...,...,...
195,31,2014,196,What is your age?
196,35,2014,197,What is your age?
197,26,2014,198,What is your age?
198,27,2014,199,What is your age?
